In [1]:
#prueba de conexion 


# Importar las librerías necesarias
import pandas as pd
import sqlalchemy as sa
from dwh_secrets import username, password
# Configuración del servidor y base de datos
DRIVERNAME = 'ODBC Driver 18 for SQL Server'
SERVERNAME = 'wdbanal-dwhp01.hdicolombia.com.co'
DATABASE = 'Liberty'
PORT = '1433'
def test_connection():
    try:
       # Configuración de la conexión
        connection_string = sa.engine.url.URL.create(
            drivername="mssql+pyodbc",
            username=username,
            password=password,
            host=SERVERNAME,
            port=PORT,
            database=DATABASE,
            query={"driver": DRIVERNAME, "TrustServerCertificate": "yes", "Encrypt": "yes"}
        )
        print("Intentando conectar al servidor...")
       # Crear el motor y realizar la conexión
        engine = sa.create_engine(connection_string)
        with engine.connect() as connection:
            print("Conexión establecida con éxito.")
           # Ejecutar una consulta de prueba
            query = "SELECT TOP 1 * FROM [Liberty_Pruebas_Actuaria].[Attrition].[acc_sise_appgenerali_siniestros_incurridos_fusion]"
            df = pd.read_sql(query, connection)
            print("Consulta ejecutada correctamente.")
           # Mostrar el resultado
            print("Resultado de la consulta:")
            print(df)
    except Exception as e:
        print(f"Error en la conexión o consulta: {e}")
if __name__ == "__main__":
    test_connection()

Intentando conectar al servidor...
Conexión establecida con éxito.


Consulta ejecutada correctamente.
Resultado de la consulta:
  process_id                 process_dt      id_day     compaa  \
0        138 2024-11-28 04:49:02.564414  2020-03-31  Generales   

  ramo_tcnicocdigo_sucursalplizanmero_siniestro  cdigo_ramo_tcnico  \
0                      AUTOMOVILES|1|25562|6151                3.0   

   ramo_tcnico  cdigo_ramo_comercial ramo_comercial  cdigo_sucursal  ...  \
0  AUTOMOVILES                   3.0    AUTOMOVILES             1.0  ...   

       marca              clase tipo_vehiculo   placa  repuestos  mano_obra  \
0  CHEVROLET  C70 149 MT TD 4X2          GRUA  HJI693        0.0        0.0   

   total_reparacion  modelo fec_recibo_pago_deducible  \
0               0.0  1985.0                      None   

                               detalle_del_siniestro  
0  REABIERTO PARA PAGO DE HONORARIOS.********* FA...  

[1 rows x 85 columns]


In [2]:
# Importar las librerías necesarias
import pandas as pd
import sqlalchemy as sa
from datetime import datetime, timedelta
import boto3
from dwh_secrets import username, password 
# Configuración global para DWH
DRIVERNAME = 'ODBC Driver 18 for SQL Server'
SERVERNAME = 'wdbanal-dwhp01.hdicolombia.com.co'
DATABASE = 'Liberty'
PORT = '1433'
# Configuración del bucket S3
S3_BUCKET = "hdi-sagemaker-project-co"
S3_KEY_PREFIX = "data-science/co-total-loss/hdi/"
# Fecha dinámica para el día anterior
yesterday = (datetime.today() - timedelta(days=1))
formatted_yesterday_sql = yesterday.strftime('%Y-%m-%d')  # Formato para SQL
formatted_yesterday_s3 = yesterday.strftime('%d-%m-%Y')  # Formato para S3

In [3]:
def create_engine_connection():
    """
   Crea la conexión con la base de datos usando SQLAlchemy y retorna el engine.
    """
    try:
        connection_string = sa.engine.URL.create(
            drivername="mssql+pyodbc",
            username=username,
            password=password,
            host=SERVERNAME,
            port=PORT,
            database=DATABASE,
            query={"driver": DRIVERNAME, "TrustServerCertificate": "yes", "Encrypt": "yes"}
        )
        engine = sa.create_engine(connection_string)
        print("Motor de conexión creado exitosamente jeje")
        return engine
    except Exception as e:
        print(f"Error al crear el motor de conexión: {e}")
        return None

In [4]:
engine = create_engine_connection()
 

Motor de conexión creado exitosamente jeje


In [5]:
def fetch_query_results(query):
    """
   Ejecuta una consulta SQL en la base de datos y retorna un DataFrame con los resultados.
    """
    try:
        engine = create_engine_connection()
        if engine is None:
            print("No se pudo establecer la conexión con el motor.")
            return None
        with engine.connect() as connection:
            print(f"Ejecutando la consulta: {query}")
            df = pd.read_sql(query, connection)
            print("Consulta ejecutada correctamente.")
            return df
    except Exception as e:
        print(f"Error al ejecutar la consulta: {e}")
        return None

In [ ]:
query_avisados_web = f"""
SELECT
   [detalle_del_siniestro] AS Detalle_Siniestro,
   [placa] AS Placa,
   [modelo] AS Modelo,
   [nmero_siniestro] AS Nro_Siniestro,
   [amparo_afectado] AS amparo,
   [estado] AS estado_siniestro,
   [causa_siniestro],
   [marca] AS Marca,
   [clase] AS cod_clase,
   [tipo_vehiculo] AS Tipo,
   [valor_asegurado] AS valor_asegurado_sin_acc,
   [fecha_de_aviso] AS Fecha_Ingreso
FROM [Liberty_Pruebas_Actuaria].[Attrition].[acc_sise_appgenerali_siniestros_incurridos_fusion]
"""
avisados_web = fetch_query_results(query_avisados_web)
# Verifica los datos

In [ ]:
avisados_web

In [ ]:
columns_to_keep = ['Fecha_Ingreso', 'Placa', 'Marca', 'Tipo','Modelo', 'Nro_Siniestro', 'Detalle_Siniestro']

avisados_web = avisados_web[columns_to_keep]

In [ ]:
avisados_web.reset_index(inplace = True, drop = True)

In [ ]:
avisados_web

In [ ]:
# avisados_web['Fecha_Ingreso'] = avisados_web['Fecha_Ingreso'].str#.strip()

In [ ]:
avisados_web['Fecha_Ingreso'] = pd.to_datetime(avisados_web['Fecha_Ingreso'], errors='coerce')

In [ ]:
def claims_keep_by_date(data):
    today = datetime.today()
    print(f"Today's date: {today}")
    print(f"Today's weekday: {today.weekday()}")
 
    if today.weekday() == 0:  # Monday
        date_threshold = pd.to_datetime(today.date() - timedelta(days=3))
    else:
        date_threshold = pd.to_datetime(today.date() - timedelta(days=4))
 
    start_date = pd.to_datetime(date_threshold)
 
    if today.weekday() == 0:  # Monday
        end_date = pd.to_datetime(date_threshold + timedelta(days=3))
    else:
        end_date = pd.to_datetime(date_threshold + timedelta(days=4))
 
    print(f"Date threshold for filtering: {date_threshold}")
    filtered_data = data[(data['Fecha_Ingreso'] >= start_date) & (data['Fecha_Ingreso'] < end_date)]
    return filtered_data

In [ ]:
from datetime import datetime, timedelta
avisados_web = claims_keep_by_date(avisados_web)

In [ ]:
avisados_web['Fecha_Ingreso'].value_counts(dropna = False)

In [ ]:
avisados_web.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
avisados_web

In [ ]:
avisados_web['Nro_Siniestro'] = avisados_web['Nro_Siniestro'].astype(str).str.replace(r'\.0$', '', regex=True).astype(int)

In [ ]:
avisados_web.reset_index(inplace = True, drop = True)

In [ ]:
avisados_web.head()

In [ ]:
avisados_web.to_excel("sample_data.xlsx", index=False)

In [ ]:
s3 = boto3.client('s3')

In [ ]:
import subprocess
import pandas as pd
from datetime import datetime, timedelta
# Configuración global del bucket y fechas
S3_BUCKET = "hdi-sagemaker-project-co"
S3_KEY_PREFIX = "data-science/co-total-loss/hdi/"
def get_avisados_sise_data():
    """
   Descarga el archivo AVISADOS-SISE desde S3 usando AWS CLI y retorna un DataFrame.
    """
   # Configuración de fechas
    yesterday = (datetime.today() - timedelta(days=2)).strftime('%d-%m-%Y')
    s3_file_name = f"Avisados SISE {yesterday}.xlsx"  # Nombre del archivo basado en el día anterior
    s3_file_path = f"s3://{S3_BUCKET}/{S3_KEY_PREFIX}{s3_file_name}"  # Ruta completa del archivo en S3
    local_file_path = f"/tmp/{s3_file_name}"  # Ruta local donde se guardará el archivo
    try:
       # Descargar el archivo usando AWS CLI
        print(f"Descargando el archivo: {s3_file_path}")
        subprocess.run(['aws', 's3', 'cp', s3_file_path, local_file_path], check=True)
        print(f"Archivo descargado exitosamente en: {local_file_path}")
       # Cargar el archivo en un DataFrame
        return pd.read_excel(local_file_path)
    except subprocess.CalledProcessError as e:
        print(f"Error al descargar el archivo desde S3: {e}")
        return None
    except Exception as e:
        print(f"Error al leer el archivo descargado: {e}")
        return None
# Llamada a la función
avisados_sise = get_avisados_sise_data()
# Validación: Mostrar las primeras filas si los datos se cargaron correctamente
if avisados_sise is not None:
    print("Datos de AVISADOS-SISE cargados correctamente:")
    display(avisados_sise.head())
else:
    print("No se pudieron obtener los datos de AVISADOS-SISE.")

In [ ]:
avisados_sise = pd.read_excel('Avisados SISE 15-12-2024.xlsx')

In [ ]:
avisados_sise.info()

In [ ]:
columns_to_keep_sise = ['Nombre Amparo Afectado', 'Numero de Placa', 'Nro. Siniestro', 'Numero de Poliza',
                       'Estado del siniestro', 'Nombre Causa Stro', 'Codigo de Marca del Vehiculo',
                       'Codigo de la Clase del Vehiculo', 'Codigo del Tipo de Vehiculo', 'Modelo',
                       'Valor Asegurado del Vehiculo sin Accesorios Eq',
                       'Valor del Vehiculo en la Tabla de Fasecolda Eq',	
                       'Valor Asegurado de Accesorios Eq', 'Color',
]

In [ ]:
avisados_sise = avisados_sise[columns_to_keep_sise]
avisados_sise

In [ ]:
final_merged_df = pd.merge(avisados_web, avisados_sise,
                          left_on='Nro_Siniestro', right_on='Nro. Siniestro', how='inner')

In [ ]:
final_merged_df = final_merged_df.drop_duplicates(subset='Nro_Siniestro', keep='first')

In [ ]:
final_merged_df

In [ ]:
final_merged_df.info()

In [ ]:
final_merged_df = final_merged_df.drop(columns=['Modelo_y', 'Nro. Siniestro'])

In [ ]:
final_merged_df = final_merged_df.rename(columns={
    'Nombre Amparo Afectado': 'Amparo',
    'Placa': 'placa',
    'Modelo_x': 'modelo',
    'Numero de Poliza': 'poliza',
    'Estado del siniestro': 'estado_siniestro',
    'Nombre Causa Stro': 'causa_siniestro',
    'Codigo de Marca del Vehiculo': 'cod_marca',
    'Codigo de la Clase del Vehiculo': 'cod_clase',
    'Codigo del Tipo de Vehiculo': 'cod_tipo',
    'Valor Asegurado del Vehiculo sin Accesorios Eq': 'valor_asegurado_sin_acc',
    'Valor del Vehiculo en la Tabla de Fasecolda Eq': 'valor_veh_fasecolda',
    'Valor Asegurado de Accesorios Eq': 'valor_accesorios',
    'Color': 'color',
    'Detalle_Siniestro': 'descripcion',
    'Nro_Siniestro': 'siniestro'
    
})

In [ ]:
columns_to_keep = [
    'Amparo', 'placa', 'siniestro', 'poliza', 'estado_siniestro', 
    'causa_siniestro', 'cod_marca', 'cod_clase', 'cod_tipo', 'modelo', 
    'valor_asegurado_sin_acc', 'valor_veh_fasecolda', 'valor_accesorios', 
    'color', 'descripcion'
    #'perdida_total', 'id'
]
final_merged_df = final_merged_df[columns_to_keep]

In [ ]:
final_merged_df

In [ ]:
final_merged_df.to_excel("prueba.xlsx", index=False)